In [1]:
!pip install pandas

In [60]:
import pandas as pd

df = pd.read_pickle(path='./tweets_database.pickle')
print(df.info)

<bound method DataFrame.info of                  tweet_id                   inserted  \
0     1230448891302227970 2020-02-28 15:53:59.572462   
1     1230454630376755200 2020-02-28 15:53:59.579021   
2     1230448421565276160 2020-02-28 15:53:59.583328   
3     1230570732373135361 2020-02-28 15:53:59.587158   
4     1230568247503859714 2020-02-28 15:53:59.591639   
...                   ...                        ...   
5280  1231761401359634432 2020-02-28 15:54:22.805641   
5281  1231796141781422082 2020-02-28 15:54:22.810084   
5282  1231801929300402176 2020-02-28 15:54:22.814195   
5283  1231803436087357440 2020-02-28 15:54:22.818355   
5284  1231801519818858497 2020-02-28 15:54:22.822186   

                          created_at          user_id_str user_screen_name  \
0     Thu Feb 20 11:07:30 +0000 2020            197351613   LOGISTIK_HEUTE   
1     Thu Feb 20 11:30:19 +0000 2020             14674814     boerseonline   
2     Thu Feb 20 11:05:38 +0000 2020            357072283    

In [73]:
# create a DF of nodes of the network
columns = ['user_id', 'username']
nodelist = pd.DataFrame(columns=columns)

# insert all tweet.users
nodelist['user_id'] = df['user_id_str']
nodelist['username'] = df['user_screen_name']
nodelist.set_index('user_id')

def get_users(user_id, username):
    reduce = df.dropna(subset=[user_id, username])
    insert_list = []
    for index, row in reduce.iterrows():
        user = {'user_id': row[user_id], 'username': row[username]}
        insert_list.append(user)
    return pd.DataFrame(insert_list, columns=columns)

# insert all users of tweet.user_mentions
nodelist = nodelist.append(get_users('user_mention_id_str', 'user_mention_screen_name'), sort=False)

# insert all users of tweet.replies
nodelist = nodelist.append(get_users('in_reply_to_user_id_str', 'in_reply_to_screen_name'), sort=False)

# insert all users of tweet.retweets
nodelist = nodelist.append(get_users('retweeted_user_id_str', 'retweeted_user_screen_name'), sort=False)

# drop duplicates
nodelist.drop_duplicates(subset=columns, inplace=True)
print(nodelist.info)

# rename columns for Gephi
gephi_nodes = nodelist.rename(columns={'user_id': 'Id', 'username': 'Label'})
gephi_nodes.to_csv('nodelist.csv', index=False)

<bound method DataFrame.info of                   user_id        username
0               197351613  LOGISTIK_HEUTE
1                14674814    boerseonline
2               357072283      mride80336
3     1216408198552281088    Findings2020
4      951353809955061760       2the_hill
...                   ...             ...
3853  1226140324512358400        DoschVan
3903               816653      TechCrunch
3910            106855826  MobiHealthNews
3919             15647855         useucom
3984            102452340      Everbridge

[3729 rows x 2 columns]>


In [74]:
# Create a DF of directed edges connecting the nodes of the network
columns = ['source_id', 'target_id']
edgelist = pd.DataFrame(columns=columns)

def get_interactions(source_id, target_id):
    reduce = df.dropna(subset=[source_id, target_id])
    insert_list = []
    for index, row in reduce.iterrows():
        interaction = {'source_id': row[source_id], 'target_id': row[target_id]}
        insert_list.append(interaction)
    return pd.DataFrame(insert_list, columns=columns)

# insert all user mentions, source=original, target=user mentioning another user
edgelist = edgelist.append(get_interactions(source_id='user_mention_id_str', target_id='user_id_str'))

# insert all replies, source=original, target=user replying to another user
edgelist = edgelist.append(get_interactions(source_id='in_reply_to_user_id_str', target_id='user_id_str'))

# insert all retweets, source=original, target=user retweeting another user
edgelist = edgelist.append(get_interactions(source_id='retweeted_user_id_str', target_id='user_id_str'))


edgelist.drop_duplicates(subset=['source_id', 'target_id'], inplace=True)

#TODO: remove self-references

print(edgelist.info)

# rename columns for Gephi
gephi_edges = edgelist.rename(columns={'source_id': 'Source', 'target_id': 'Target'})
gephi_edges.to_csv('edgelist.csv', index=False)

<bound method DataFrame.info of                source_id            target_id
0             3367084329  1216408198552281088
1     995604886409695233   951353809955061760
2     995604886409695233   912951056019181568
3             3367084329  1079274338111709185
4     995604886409695233           1548367159
..                   ...                  ...
109           1136810036           1136810036
110  1029084199725223937  1029084199725223937
120   845415526407028736   845415526407028736
123               636603               636603
152   852448997541326848   852448997541326848

[3510 rows x 2 columns]>
